# MSMH5 file

## Using the native MSMH5 OpenMM reporter

In [1]:
import openmm as mm
from openmm import app
from openmm import unit

import molsysmt as msm
from molsysmt import pyunitwizard as puw

In [2]:
molsys = msm.convert('4POC')

In [3]:
molsys = msm.extract(molsys, selection='molecule_type=="protein"')

In [4]:
molsys = msm.build.add_missing_hydrogens(molsys)

In [5]:
molsys = msm.build.solvate(molsys)

In [6]:
topology = msm.convert(molsys, 'openmm.Topology')

In [7]:
coordinates = msm.get(molsys, coordinates=True)

In [8]:
positions = puw.convert(coordinates[0], to_form='openmm.unit')

In [9]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')
system = forcefield.createSystem(topology, nonbondedMethod=app.PME,
        nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)
platform = mm.Platform.getPlatformByName("CPU")
simulation = app.Simulation(topology, system, integrator, platform)
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

In [ ]:
from molsysmt.thirds.openmm.reporters import H5MSMReporter, TQDMReporter

In [ ]:
tqdm_reporter = TQDMReporter(1000, 50000)
simulation.reporters.append(tqdm_reporter)

In [ ]:
msmh5_reporter = H5MSMReporter('traj.msmh5', 1000, steps=50000, selection='all',
            topology=topology, time=True, box=True, coordinates=True, velocities=False,
            potentialEnergy=True, kineticEnergy=True, temperature=True,
            includeInitialContext=True, constantReportInterval=True,
            constantStepSize=True, constantBox=True,
            auto_close=True)
simulation.reporters.append(msmh5_reporter)

In [ ]:
simulation.step(50000)

## A first glance at the msmh5 file

In [ ]:
aa = msm.convert('traj.msmh5', 'molsysmt.MSMH5FileHandler')

In [ ]:
'topology' in aa.file

In [ ]:
aa.file['topology']['atoms'].attrs['n_atoms']

In [ ]:
msm.get('traj.msmh5', n_lipids=True)

## Extract a new msmh5 file